In [36]:
# Import libraries

from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [12]:
# import the data

NVR = pdr.get_data_yahoo('NVR',start='2017-01-01',end='2021-12-31')
NVR

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,1650.000000,1665.000000,1631.780029,1649.989990,1649.989990,37400
2017-01-04,1654.000000,1686.520020,1645.030029,1678.770020,1678.770020,22500
2017-01-05,1676.199951,1690.829956,1671.510010,1689.400024,1689.400024,24200
2017-01-06,1684.010010,1697.479980,1675.010010,1678.099976,1678.099976,23400
2017-01-09,1680.800049,1689.000000,1666.550049,1681.500000,1681.500000,17100
...,...,...,...,...,...,...
2021-12-23,5826.669922,5847.799805,5779.890137,5784.629883,5784.629883,7400
2021-12-27,5817.000000,5830.970215,5763.479980,5820.069824,5820.069824,12600
2021-12-28,5820.500000,5893.500000,5820.500000,5869.470215,5869.470215,7900


In [15]:
# Creating a new df and using only the adj. close price
df = NVR[['Adj Close']]
df

,Adj Close
Date,
2017-01-03,1649.989990
2017-01-04,1678.770020
2017-01-05,1689.400024
2017-01-06,1678.099976
2017-01-09,1681.500000
...,...
2021-12-23,5784.629883
2021-12-27,5820.069824
2021-12-28,5869.470215


In [17]:
# creating a variable for predicting stock price for n number of days into the future
forecast_out = 30
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)
df

<ipython-input-17-09ab9223795b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prediction'] = df[['Adj Close']].shift(-forecast_out)


,Adj Close,Prediction
Date,,
2017-01-03,1649.989990,1893.310059
2017-01-04,1678.770020,1897.319946
2017-01-05,1689.400024,1896.349976
2017-01-06,1678.099976,1922.199951
2017-01-09,1681.500000,1923.459961
...,...,...
2021-12-23,5784.629883,NaN
2021-12-27,5820.069824,NaN
2021-12-28,5869.470215,NaN


In [21]:
# Creating the independent dataset (X) and convertng to a numpy array
X = np.array(df.drop(['Prediction'],1))

# Removing the last n rows to remove NaNs
X = X[:-forecast_out]
X

array([[1649.98999023],
       [1678.77001953],
       [1689.40002441],
       ...,
       [5253.81005859],
       [5277.97998047],
       [5166.        ]])

In [22]:
# Creating the dependent dataset (y) and convertig to a numpy array
y = np.array(df['Prediction'])

# Getting all the y values except the last n rows, to remove NaNs
y = y[:-forecast_out]
y

array([1893.31005859, 1897.31994629, 1896.34997559, ..., 5869.47021484,
       5959.33007812, 5906.02978516])

In [23]:
# splitting the data into training and testing set (80/20)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [24]:
# Creating and training the SVM model (Regressor)
svr_rbf = SVR(kernel = 'rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)


SVR(C=1000.0, gamma=0.1)

In [25]:
# Testing the model: score returns the coeff. of determination R^2 of predictions
svm_confidence = svr_rbf.score(x_test,y_test)
svm_confidence

0.5916528221159867

In [26]:
# Creating and training the linear regression model
lr = LinearRegression()

# Training the model
lr.fit(x_train,y_train)

LinearRegression()

In [28]:
# Testing the model: score returns the coeff. of determination R^2 of predictions
lr_confidence = lr.score(x_test,y_test)
lr_confidence

0.85637924633266

In [47]:
# Creating and training the random forest model
rf = RandomForestRegressor(n_estimators=50,max_depth=3)

# Training the model
rf.fit(x_train,y_train)

RandomForestRegressor(max_depth=3, n_estimators=50)

In [48]:
# Testing the model: score returns the coeff. of determination R^2 of predictions
rf_confidence = rf.score(x_test,y_test)
rf_confidence

0.8812561032267651

In [30]:
# Setting x_test (forecast) for last 30 rows using Adj. Close
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
x_forecast

array([[5160.        ],
       [5109.99023438],
       [5216.00976562],
       [5210.81982422],
       [5320.        ],
       [5351.39990234],
       [5291.10009766],
       [5281.72998047],
       [5225.33984375],
       [5404.56005859],
       [5549.18017578],
       [5480.85986328],
       [5626.14013672],
       [5671.37988281],
       [5892.58007812],
       [5863.52001953],
       [5935.72021484],
       [5785.89990234],
       [5832.04980469],
       [5907.37011719],
       [5760.35009766],
       [5636.77978516],
       [5602.74023438],
       [5747.04980469],
       [5813.37988281],
       [5784.62988281],
       [5820.06982422],
       [5869.47021484],
       [5959.33007812],
       [5906.02978516]])

In [32]:
# Printing SVM predictions for next n days
svr_prediction = svr_rbf.predict(x_forecast)
svr_prediction

array([4494.50108391, 4501.55267533, 3924.2041884 , 4154.23244617,
       3535.05950755, 3519.05229642, 3700.71688685, 3519.05230858,
       4639.89880919, 3519.05229642, 3519.05229642, 3519.05229642,
       3519.05229642, 3519.05229642, 3519.05229642, 3519.05229642,
       3519.05229642, 3519.05229642, 3519.05229642, 3519.05229642,
       3519.05229642, 3519.05229642, 3519.05229642, 3519.05229642,
       3519.05229642, 3519.05229642, 3519.05229642, 3519.05229642,
       3519.05229642, 3519.05229642])

In [31]:
# Printing LR predictions for next n days
lr_prediction = lr.predict(x_forecast)
lr_prediction

array([5170.93746163, 5123.44497513, 5224.12793362, 5219.19923182,
       5322.88374143, 5352.7031061 , 5295.43853738, 5286.54007208,
       5232.98837526, 5403.18740594, 5540.52796088, 5475.64660264,
       5613.61408437, 5656.57665386, 5866.64257116, 5839.04527245,
       5907.61121669, 5765.33222217, 5809.15913451, 5880.68814249,
       5741.0684861 , 5623.71817811, 5591.39203369, 5728.43767337,
       5791.42897718, 5764.12613003, 5797.78217536, 5844.69596021,
       5930.03265976, 5879.41527708])

In [49]:
# Printing LR predictions for next n days
rf_prediction = rf.predict(x_forecast)
rf_prediction

array([5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995, 5091.39293995, 5091.39293995,
       5091.39293995, 5091.39293995])

In [34]:
NVR['Adj Close']

Date
2017-01-03    1649.989990
2017-01-04    1678.770020
2017-01-05    1689.400024
2017-01-06    1678.099976
2017-01-09    1681.500000
                 ...     
2021-12-23    5784.629883
2021-12-27    5820.069824
2021-12-28    5869.470215
2021-12-29    5959.330078
2021-12-30    5906.029785
Name: Adj Close, Length: 1258, dtype: float64

In [33]:
# visualizing the data
plt.figure(figsize=(16,9))
plt.plot(NVR['Adj Close'], label = 'NVR')
plt.plot(lr_prediction, label = 'LR')
plt.title('NVR Adj. Close Price History')
plt.xlabel('Jan 03 2017 to Dec 31 2021')
plt.ylabel('Adj. Close Price USD ($)')
plt.legend(loc='upper left')
plt.show()

ValueError: view limit minimum -36907.700000000004 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 1152x648 with 1 Axes>